Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in `1_notmnist.ipynb`.

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

In [4]:
train_subset = 10000
beta = 0.01
num_steps = 801

In [5]:
graph = tf.Graph()
with graph.as_default():
    tf_train_dataset = tf.constant(train_dataset[:train_subset,:])
    tf_train_labels = tf.constant(train_labels[:train_subset])
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    W = tf.Variable(tf.truncated_normal([image_size*image_size,num_labels]))
    b = tf.Variable(tf.zeros(num_labels))
    logits = tf.matmul(tf_train_dataset,W) + b 
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits,labels=tf_train_labels))
    regularizer=tf.nn.l2_loss(W)
    loss = tf.reduce_mean(loss+beta*regularizer)
    optimizer = tf.train.AdamOptimizer(0.01).minimize(loss)   
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(tf.matmul(tf_valid_dataset,W) + b)
    test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset,W) + b)

In [6]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])
with tf.Session(graph=graph) as sess:
    tf.initialize_all_variables().run()
    print('Initialized!!!')
    for step in range(num_steps):
        _,l,predictions = sess.run([optimizer,loss,train_prediction])
        if step % 100==0:
            print('loss as step {}: {}'.format(step,l))
            print('training accuracy: {:.1f}'.format(accuracy(predictions,train_labels[:train_subset,:])))
            print('Validation accuracy: {:.1f}'.format(accuracy(valid_prediction.eval(), 
                                                           valid_labels)))
    print('Test accuracy: {:.1f}'.format(accuracy(test_prediction.eval(), test_labels))) 

Instructions for updating:
Use `tf.global_variables_initializer` instead.
Initialized!!!
loss as step 0: 47.77642059326172
training accuracy: 8.2
Validation accuracy: 9.6
loss as step 100: 8.613282203674316
training accuracy: 73.0
Validation accuracy: 71.0
loss as step 200: 1.9286702871322632
training accuracy: 81.7
Validation accuracy: 79.1
loss as step 300: 0.8498624563217163
training accuracy: 84.7
Validation accuracy: 81.6
loss as step 400: 0.7004004716873169
training accuracy: 84.9
Validation accuracy: 81.9
loss as step 500: 0.6812186241149902
training accuracy: 85.0
Validation accuracy: 82.0
loss as step 600: 0.6788465976715088
training accuracy: 85.0
Validation accuracy: 81.9
loss as step 700: 0.6785584688186646
training accuracy: 85.0
Validation accuracy: 82.0
loss as step 800: 0.6785227060317993
training accuracy: 85.0
Validation accuracy: 82.0
Test accuracy: 88.8


In [7]:
num_nodes = 1024
batch_size = 128
beta = 0.01
num_steps = 3001

In [8]:
graph = tf.Graph()
with graph.as_default():
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size,image_size*image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size,num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    W1 = tf.Variable(tf.truncated_normal([image_size*image_size,num_nodes],stddev=np.sqrt(2.0/float(image_size*image_size))))
    b1 = tf.Variable(tf.zeros([num_nodes]))
    W2 = tf.Variable(tf.truncated_normal([num_nodes,num_labels],stddev=np.sqrt(2.0/float(num_nodes))))                 
    b2 = tf.Variable(tf.zeros([num_labels]))                 
    log1 = tf.matmul(tf_train_dataset,W1) + b1
    rel = tf.nn.relu(log1)
    log2 = tf.matmul(rel,W2) +b2
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=log2,labels=tf_train_labels))
    regularizers = tf.nn.l2_loss(W1)+tf.nn.l2_loss(W2)
    loss = tf.reduce_mean(loss+beta*regularizers)
    optimizer = tf.train.AdamOptimizer(0.01).minimize(loss)
    train_prediction = tf.nn.softmax(log2)
    logi1=tf.matmul(tf_valid_dataset,W1)+b1
    reli = tf.nn.relu(logi1)
    logi2 = tf.matmul(reli,W2)+b2
    valid_prediction = tf.nn.softmax(logi2)
    logit1 = tf.matmul(tf_test_dataset,W1)+b1
    relite = tf.nn.relu(logit1)
    logit2 = tf.matmul(relite,W2)+b2
    test_prediction = tf.nn.softmax(logit2)
  

In [9]:
with tf.Session(graph=graph) as sess:
    tf.global_variables_initializer().run()
    print('initialized!!!')
    for step in range(num_steps):
        offset = (step*batch_size)%(train_labels.shape[0]-batch_size)
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = sess.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        if step % 100 == 0:
            print("Minibatch loss at step {}: {}".format(step, l))
            print("Minibatch accuracy: {:.1f}".format(accuracy(predictions, batch_labels)))
            print("Validation accuracy: {:.1f}".format(accuracy(valid_prediction.eval(), valid_labels)))
    print("Test accuracy: {:.1f}".format(accuracy(test_prediction.eval(), test_labels)))

initialized!!!
Minibatch loss at step 0: 10.396864891052246
Minibatch accuracy: 7.0
Validation accuracy: 44.0
Minibatch loss at step 100: 1.2093873023986816
Minibatch accuracy: 77.3
Validation accuracy: 78.1
Minibatch loss at step 200: 1.0718624591827393
Minibatch accuracy: 80.5
Validation accuracy: 78.4
Minibatch loss at step 300: 1.0656440258026123
Minibatch accuracy: 76.6
Validation accuracy: 77.8
Minibatch loss at step 400: 1.1450797319412231
Minibatch accuracy: 71.9
Validation accuracy: 78.8
Minibatch loss at step 500: 0.744419515132904
Minibatch accuracy: 85.2
Validation accuracy: 80.6
Minibatch loss at step 600: 0.9839335680007935
Minibatch accuracy: 80.5
Validation accuracy: 77.9
Minibatch loss at step 700: 0.947108268737793
Minibatch accuracy: 77.3
Validation accuracy: 80.3
Minibatch loss at step 800: 0.6904222369194031
Minibatch accuracy: 85.2
Validation accuracy: 80.8
Minibatch loss at step 900: 0.8969485759735107
Minibatch accuracy: 78.9
Validation accuracy: 79.8
Minibatch 

---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [10]:
train_dataset_2=train_dataset[:500,:]
train_labels_2=train_labels[:500]
with tf.Session(graph=graph) as sess:
    tf.global_variables_initializer().run()
    print('intialized....')
    for step in range(num_steps):
        offset = (step*batch_size)%(train_labels_2.shape[0]-batch_size)
        batch_data = train_dataset_2[offset:(offset+batch_size),:]
        batch_labels=train_labels_2[offset:(offset+batch_size),:]
        feed_dict = {tf_train_dataset:batch_data,tf_train_labels:batch_labels}
        _,l,predictions = sess.run([optimizer,loss,train_prediction],feed_dict=feed_dict)
        if (step% 100 == 0):
            print("Minibatch loss at step {}: {}".format(step, l))
            print("Minibatch accuracy: {:.1f}".format(accuracy(predictions, batch_labels)))
            print("Validation accuracy: {:.1f}".format(accuracy(valid_prediction.eval(), valid_labels)))
    print("Test accuracy: {:.1f}".format(accuracy(test_prediction.eval(), test_labels)))

intialized....
Minibatch loss at step 0: 10.473159790039062
Minibatch accuracy: 6.2
Validation accuracy: 41.9
Minibatch loss at step 100: 0.5851584672927856
Minibatch accuracy: 96.1
Validation accuracy: 74.0
Minibatch loss at step 200: 0.5020344257354736
Minibatch accuracy: 97.7
Validation accuracy: 76.0
Minibatch loss at step 300: 0.3500511348247528
Minibatch accuracy: 99.2
Validation accuracy: 78.4
Minibatch loss at step 400: 0.3796970248222351
Minibatch accuracy: 96.9
Validation accuracy: 76.0
Minibatch loss at step 500: 0.47753533720970154
Minibatch accuracy: 93.0
Validation accuracy: 75.9
Minibatch loss at step 600: 0.3682549297809601
Minibatch accuracy: 98.4
Validation accuracy: 77.1
Minibatch loss at step 700: 0.3127317726612091
Minibatch accuracy: 100.0
Validation accuracy: 79.0
Minibatch loss at step 800: 0.2919920086860657
Minibatch accuracy: 100.0
Validation accuracy: 77.7
Minibatch loss at step 900: 0.2679128050804138
Minibatch accuracy: 100.0
Validation accuracy: 78.0
Mini

---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [12]:
with graph.as_default():
    tf_train_dataset = tf.placeholder(tf.float32,shape = (batch_size,image_size*image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape = (batch_size,num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    W1 = tf.Variable(tf.truncated_normal([image_size*image_size,num_nodes]))
    b1 = tf.Variable(tf.zeros([num_nodes]))
    W2 = tf.Variable(tf.truncated_normal([num_nodes,num_labels]))
    b2 = tf.Variable(tf.zeros([num_labels]))
    log1 = tf.matmul(tf_train_dataset,W1) + b1
    rel = tf.nn.relu(log1)
    keep_prob = tf.placeholder('float')
    rel_dropout = tf.nn.dropout(rel,keep_prob)
    log2 = tf.matmul(rel_dropout,W2) + b2
    regularizers = tf.nn.l2_loss(W1)+tf.nn.l2_loss(W2)
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=log2,labels=tf_train_labels))
    loss = tf.reduce_mean(loss+beta*regularizers)
    optimizer = tf.train.GradientDescentOptimizer(0.01).minimize(loss)
    train_prediction = tf.nn.softmax(log2)
    logi1 = tf.matmul(tf_valid_dataset,W1)+b1
    reli = tf.nn.relu(logi1)
    logi2 = tf.matmul(reli,W2) + b2
    valid_prediction = tf.nn.softmax(logi2)
    logit1 = tf.matmul(tf_test_dataset,W1)+b1
    relit = tf.nn.relu(logit1)
    logit2 = tf.matmul(relit,W2)+b2
    test_prediction = tf.nn.softmax(logit2) 


In [13]:
num_steps = 3001
with tf.Session(graph=graph) as sess:
    tf.global_variables_initializer().run()
    print('initialized!!!!!!')
    for step in range (num_steps):
        offset = (step*batch_size)%(train_labels.shape[0]-batch_size)
        batch_data = train_dataset[offset:(offset+batch_size),:]
        batch_labels = train_labels[offset:(offset+batch_size),:]
        feed_dict = {tf_train_dataset:batch_data,tf_train_labels:batch_labels, keep_prob:0.5}
        _,l,predictions = sess.run([optimizer,loss,train_prediction],feed_dict=feed_dict)
        if (step % 500 == 0):
            print("Minibatch loss at step {}: {}".format(step, l))
            print("Minibatch accuracy: {:.1f}".format(accuracy(predictions, batch_labels)))
            print("Validation accuracy: {:.1f}".format(accuracy(valid_prediction.eval(), valid_labels)))
    print("Test accuracy: {:.1f}".format(accuracy(test_prediction.eval(), test_labels)))

initialized!!!!!!
Minibatch loss at step 0: 3681.73046875
Minibatch accuracy: 7.8
Validation accuracy: 13.5
Minibatch loss at step 500: 2883.648193359375
Minibatch accuracy: 76.6
Validation accuracy: 79.8
Minibatch loss at step 1000: 2618.397216796875
Minibatch accuracy: 73.4
Validation accuracy: 81.3
Minibatch loss at step 1500: 2347.4150390625
Minibatch accuracy: 80.5
Validation accuracy: 82.3
Minibatch loss at step 2000: 2117.76220703125
Minibatch accuracy: 81.2
Validation accuracy: 82.6
Minibatch loss at step 2500: 1915.7890625
Minibatch accuracy: 75.8
Validation accuracy: 83.0
Minibatch loss at step 3000: 1729.4674072265625
Minibatch accuracy: 73.4
Validation accuracy: 83.2
Test accuracy: 90.4


In [14]:
# extreme overfitting
num_steps = 3001
beta = 0.01
train_dataset_2 = train_dataset[:500, :]
train_labels_2 = train_labels[:500]
with tf.Session(graph=graph) as sess:
    tf.global_variables_initializer().run()
    print ('initialized?')
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels_2.shape[0] - batch_size)
        batch_data = train_dataset_2[offset:(offset + batch_size), :]
        batch_labels = train_labels_2[offset:(offset + batch_size), :]
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels,  keep_prob : 0.5}
        _,l,predictions = sess.run([optimizer,loss,train_prediction],feed_dict=feed_dict)
        if step % 500 == 0:
            print("Minibatch loss at step {}: {}".format(step, l))
            print("Minibatch accuracy: {:.1f}".format(accuracy(predictions, batch_labels)))
            print("Validation accuracy: {:.1f}".format(accuracy(valid_prediction.eval(), valid_labels)))
    print("Test accuracy: {:.1f}".format(accuracy(test_prediction.eval(), test_labels)))


initialized?
Minibatch loss at step 0: 3658.58544921875
Minibatch accuracy: 9.4
Validation accuracy: 10.6
Minibatch loss at step 500: 2877.218017578125
Minibatch accuracy: 79.7
Validation accuracy: 76.5
Minibatch loss at step 1000: 2578.45068359375
Minibatch accuracy: 91.4
Validation accuracy: 77.3
Minibatch loss at step 1500: 2325.86669921875
Minibatch accuracy: 97.7
Validation accuracy: 77.8
Minibatch loss at step 2000: 2103.765869140625
Minibatch accuracy: 97.7
Validation accuracy: 77.9
Minibatch loss at step 2500: 1903.343017578125
Minibatch accuracy: 97.7
Validation accuracy: 77.9
Minibatch loss at step 3000: 1723.283447265625
Minibatch accuracy: 96.9
Validation accuracy: 78.0
Test accuracy: 86.0


---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


In [60]:
import math as math
batch_size = 128
beta = 0.001

hidden_nodes_1 = 1024
hidden_nodes_2 = int(hidden_nodes_1 * 0.5)
hidden_nodes_3 = int(hidden_nodes_1 * np.power(0.5, 2))
hidden_nodes_4 = int(hidden_nodes_1 * np.power(0.5, 3))
hidden_nodes_5 = int(hidden_nodes_1 * np.power(0.5, 4))

graph = tf.Graph()
with graph.as_default():
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    weights_1 = tf.Variable(tf.truncated_normal([image_size * image_size, hidden_nodes_1], stddev=math.sqrt(2.0/(image_size*image_size))))
    biases_1 = tf.Variable(tf.zeros([hidden_nodes_1]))
    weights_2 = tf.Variable(tf.truncated_normal([hidden_nodes_1, hidden_nodes_2], stddev=math.sqrt(2.0/hidden_nodes_1)))
    biases_2 = tf.Variable(tf.zeros([hidden_nodes_2]))
    weights_3 = tf.Variable(tf.truncated_normal([hidden_nodes_2, hidden_nodes_3], stddev=math.sqrt(2.0/hidden_nodes_2)))
    biases_3 = tf.Variable(tf.zeros([hidden_nodes_3]))
    weights_4 = tf.Variable(tf.truncated_normal([hidden_nodes_3, hidden_nodes_4], stddev=math.sqrt(2.0/hidden_nodes_3)))
    biases_4 = tf.Variable(tf.zeros([hidden_nodes_4]))
    weights_5 = tf.Variable(tf.truncated_normal([hidden_nodes_4, hidden_nodes_5], stddev=math.sqrt(2.0/hidden_nodes_4)))
    biases_5 = tf.Variable(tf.zeros([hidden_nodes_5]))
    weights_6 = tf.Variable(tf.truncated_normal([hidden_nodes_5, num_labels], stddev=math.sqrt(2.0/hidden_nodes_5)))
    biases_6 = tf.Variable(tf.zeros([num_labels]))

    logits_1 = tf.matmul(tf_train_dataset, weights_1) + biases_1
    hidden_layer_1 = tf.nn.relu(logits_1)

    keep_prob = tf.placeholder("float")
    hidden_layer_1_dropout = tf.nn.dropout(hidden_layer_1, keep_prob)
    
    

    logits_2 = tf.matmul(hidden_layer_1_dropout, weights_2) + biases_2
    hidden_layer_2 = tf.nn.relu(logits_2)
 
    hidden_layer_2_dropout = tf.nn.dropout(hidden_layer_2, keep_prob)
    

    logits_3 = tf.matmul(hidden_layer_2_dropout, weights_3) + biases_3
    hidden_layer_3 = tf.nn.relu(logits_3)
 
    hidden_layer_3_dropout = tf.nn.dropout(hidden_layer_3, keep_prob)
    

    logits_4 = tf.matmul(hidden_layer_3_dropout, weights_4) + biases_4
    hidden_layer_4 = tf.nn.relu(logits_4)


    hidden_layer_4_dropout = tf.nn.dropout(hidden_layer_4, keep_prob)
    

    logits_5 = tf.matmul(hidden_layer_4_dropout, weights_5) + biases_5
    hidden_layer_5 = tf.nn.relu(logits_5)

    hidden_layer_5_dropout = tf.nn.dropout(hidden_layer_5, keep_prob)
    

    logits_6 = tf.matmul(hidden_layer_5_dropout, weights_6) + biases_6 
    

    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits_6,labels= tf_train_labels))

    regularizers = tf.nn.l2_loss(weights_1) + tf.nn.l2_loss(weights_2) + \
                   tf.nn.l2_loss(weights_3) + tf.nn.l2_loss(weights_4) + \
                   tf.nn.l2_loss(weights_5) + tf.nn.l2_loss(weights_6)
    loss = tf.reduce_mean(loss + beta * regularizers)
    
    global_step = tf.Variable(0)  
    start_learning_rate = 0.5
    learning_rate = tf.train.exponential_decay(start_learning_rate, global_step, 100000, 0.96, staircase=True)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)

    train_prediction = tf.nn.softmax(logits_6)
    

    valid_logits_1 = tf.matmul(tf_valid_dataset, weights_1) + biases_1
    valid_relu_1 = tf.nn.relu(valid_logits_1)
    
    valid_logits_2 = tf.matmul(valid_relu_1, weights_2) + biases_2
    valid_relu_2 = tf.nn.relu(valid_logits_2)
    
    valid_logits_3 = tf.matmul(valid_relu_2, weights_3) + biases_3
    valid_relu_3 = tf.nn.relu(valid_logits_3)
    
    valid_logits_4 = tf.matmul(valid_relu_3, weights_4) + biases_4
    valid_relu_4 = tf.nn.relu(valid_logits_4)
    
    valid_logits_5 = tf.matmul(valid_relu_4, weights_5) + biases_5
    valid_relu_5 = tf.nn.relu(valid_logits_5)
    
    valid_logits_6 = tf.matmul(valid_relu_5, weights_6) + biases_6
    
    valid_prediction = tf.nn.softmax(valid_logits_6)

    test_logits_1 = tf.matmul(tf_test_dataset, weights_1) + biases_1
    test_relu_1 = tf.nn.relu(test_logits_1)
    
    test_logits_2 = tf.matmul(test_relu_1, weights_2) + biases_2
    test_relu_2 = tf.nn.relu(test_logits_2)
    
    test_logits_3 = tf.matmul(test_relu_2, weights_3) + biases_3
    test_relu_3 = tf.nn.relu(test_logits_3)
    
    test_logits_4 = tf.matmul(test_relu_3, weights_4) + biases_4
    test_relu_4 = tf.nn.relu(test_logits_4)
    
    test_logits_5 = tf.matmul(test_relu_4, weights_5) + biases_5
    test_relu_5 = tf.nn.relu(test_logits_5)
    
    test_logits_6 = tf.matmul(test_relu_5, weights_6) + biases_6
    
    test_prediction = tf.nn.softmax(test_logits_6)
    
                           
                           




In [62]:
num_steps = 15000

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print("Initialized")
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, keep_prob : 0.5}
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print("Minibatch loss at step {}: {}".format(step, l))
            print("Minibatch accuracy: {:.1f}".format(accuracy(predictions, batch_labels)))
            print("Validation accuracy: {:.1f}".format(accuracy(valid_prediction.eval(), valid_labels)))
    print("Test accuracy: {:.1f}".format(accuracy(test_prediction.eval(), test_labels)))

Initialized
Minibatch loss at step 0: 5.232653617858887
Minibatch accuracy: 6.2
Validation accuracy: 12.6
Minibatch loss at step 500: 1.6331731081008911
Minibatch accuracy: 84.4
Validation accuracy: 81.8
Minibatch loss at step 1000: 1.4125897884368896
Minibatch accuracy: 82.0
Validation accuracy: 83.5
Minibatch loss at step 1500: 1.2339909076690674
Minibatch accuracy: 82.0
Validation accuracy: 82.0
Minibatch loss at step 2000: 0.8745301365852356
Minibatch accuracy: 89.1
Validation accuracy: 84.9
Minibatch loss at step 2500: 1.0069442987442017
Minibatch accuracy: 83.6
Validation accuracy: 84.5
Minibatch loss at step 3000: 0.9116887450218201
Minibatch accuracy: 81.2
Validation accuracy: 85.1
Minibatch loss at step 3500: 0.880215585231781
Minibatch accuracy: 83.6
Validation accuracy: 85.1
Minibatch loss at step 4000: 0.7117215394973755
Minibatch accuracy: 90.6
Validation accuracy: 85.8
Minibatch loss at step 4500: 0.7308399677276611
Minibatch accuracy: 87.5
Validation accuracy: 85.2
Minib